In [5]:
# Load packages
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from models import NeuralNetwork, evaluate_model, TrainConfig
from utils import load_data, split_data, create_dataset

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name())

**Overview of the flow:**
1. Load the raw data in to a RawParliamentData object containing (text ID, speaker ID, text, label)
2. Split the raw data into train, dev, test datasets. Data is split so that speakers in each set does not appear in another set.
3. Prepare a TfidfVectorizer. Fit the vectorizer on the train set, and use it to transform all train, dev, test sets. Use the `create_dataset()` function on the RawParliamentData objects, and supply the fitted encoder so that the same trained encoder is used on all sets.
4. Run the `train_neural_network()` function.

**To test different types of train-dev-test sets:**
If you want to use some countries as the **train set**, and some other countries as the **dev & test set**, you will need to load the train, dev, test countries separately. For example: 

```python
train_raw = load_data(folder_path="data/power/", file_list=['power-gb-train.tsv',],text_head='text_en')
dev_raw = load_data(folder_path="data/power/", file_list=['power-ua-train.tsv',],text_head='text_en')
test_raw = load_data(folder_path="data/power/", file_list=['power-cz-train.tsv',],text_head='text_en')
```

In [6]:

#%%
file_list = [
    'power-gb-train.tsv',
    'power-ua-train.tsv'
]

full_data = load_data(folder_path="data/power/", file_list=file_list,text_head='text_en')
train_dev_raw, test_raw = split_data(full_data, test_size=0.2, random_state=0)
train_raw, dev_raw = split_data(train_dev_raw, test_size=0.2, random_state=0)


Load power-gb-train.tsv...
Load power-ua-train.tsv...


In [7]:

#%%
print("Prepare data encoder...")
train_encoder = TfidfVectorizer(sublinear_tf=True, analyzer="char", ngram_range=(1,3))
train_encoder.fit(train_raw.texts_)

print("Prepare data...")
train_dataset = create_dataset(train_raw, train_encoder)
dev_dataset = create_dataset(dev_raw, train_encoder)
test_dataset = create_dataset(test_raw, train_encoder)


Prepare data encoder...
Prepare data...


Train model.
If you use Google Colab or your machine has a CUDA-supported graphic card, you can try setting `device='cuda'`

In [10]:
train_config = TrainConfig(
    batch_size      = 128,
    num_epochs      = 20,
    early_stop      = True,
    violation_limit = 5
)

model = NeuralNetwork(
    input_size=len(train_encoder.vocabulary_),
    num_classes=2,
    hidden_size=64,
    device='cpu'
)

model.fit(
    train_dataset   = train_dataset,
    dev_dataset     = dev_dataset,
    train_config    = train_config
)


Epoch 7: 100%|██████████| 222/222 [00:02<00:00, 83.46batch/s]


Validation loss did not improve for 5 epochs. Stopping early.


NeuralNetwork(
  (fc1): Linear(in_features=32187, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

Check model's final performance

In [11]:
precision, recall, f1 = evaluate_model(model, test_dataset)
print(precision, recall, f1)



0.7274233867150233 0.7233966798943638 0.7245857594048347
